In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
# import autokeras as ak
import keras
from sklearn.preprocessing import MinMaxScaler
import time
from matplotlib.animation import FuncAnimation
from joblib import dump, load
from pandas.core.computation.check import NUMEXPR_INSTALLED
import joblib

In [11]:
class ExtruderController():
    def __init__(self,screwspeed,solidcontent,massflow_ref,Energy_ref,Torque_ref):
        self.z = 0
        self.x = 1 
        self.screwspeed_epsilon = 20          #80
        self.screwspeed_etha = 20             #80
        self.solidcontent_epsilon= 1          #2.5
        self.solidcontent_etha = 1            #2.5
#get Data
        self.screwspeed=screwspeed            # float(input("enter the value of screwspeed"))
        self.solidcontent=solidcontent       #float(input("enter the value of solidcontent"))
        self.massflow_ref=massflow_ref       #float(input("enter the value of massflow_ref"))
        self.Energy_ref=Energy_ref           #float(input("enter the desired value of Energy_ref"))
        self.Torque_ref=Torque_ref           #float(input("enter the desired value of Torque_ref"))
        self.Energy_list=[]
        self.Torque_list=[]
        self.screwspeed_list=[]
        self.solidcontent_list=[]
        self.model_Energy=joblib.load('model_Energy.h5')  
        self.model_Torque=joblib.load('model_Torque.h5')  
        self.model_cm_Neurocontroller=joblib.load('model_cm_Neurocontroller.h5')  
        self.model_Dn_Neurocontroller=joblib.load('model_Dn_Neurocontroller.h5')  
        self.scaler_ext_input = load("scaler_ext.joblib")
        self.scaler_nc_input = load("scaler_nc.joblib")
            
#unitlity function --> dont get self --> @staticmethod
#Extruder function get [masflow_ref,screwspeed,solidcontent] and return [self.Energy_pred, self.Torque_pred]  
    def Extruder(self):
        print('inside Extruder')
        print(f'screwspeed:{self.screwspeed:}, solidcontent:{self.solidcontent:}')
#prepare inputs 
        input_Extruder=self.scaler_ext_input.transform(np.array([[self.massflow_ref,self.screwspeed,self.solidcontent]], dtype=object)  )
        self.screwspeed_list.append(self.screwspeed)
        self.solidcontent_list.append(self.solidcontent)       
#predict Energy-pred
        
        self.Energy_pred = self.model_Energy.predict(input_Extruder)
        self.Energy_list.append(self.Energy_pred)    
#predict Torque_pred
        
        self.Torque_pred = self.model_Torque.predict(input_Extruder)
        self.Torque_list.append(self.Torque_pred)
        print(self.Energy_pred, self.Torque_pred)
              
#         fig,ax=plt.subplots(2,2,figsize=(10,10))
#         self.line,=ax[0,0].plot(self.screwspeed,Energy)

#     def animation_frame():
#         self.screwspeed_list.append(self.screwspeed)
#         self.Energy_list.append(self.Energy_pred)
#         self.line.set_xdata()
#         self.line.set_ydata()
#         return self.line,
    
# check Convergence:
    def coarse_criterion(self):
        print('inside coarse criterion')
        if self.z==0:
            if 0.9*self.Energy_ref<self.Energy_pred<1.1*self.Energy_ref and 0.9*self.Torque_ref<self.Torque_pred<1.1*self.Torque_ref:
                self.screwspeed_fin=self.screwspeed
                self.solidcontent_fin=self.solidcontent
                self.z=1
                return True
            else:
                return False   
        else:#self.z!=0
            return True
                    
    def fine_criterion(self):
        print('inside fine criterion')
        if 0.99*self.Energy_ref<self.Energy_pred<1.01*self.Energy_ref and 0.99*self.Torque_ref<self.Torque_pred<1.01*self.Torque_ref:
            print("everything is in the required range!")
            return True 
        else:
            return False 
        
        
    def screwspeed_update(self):
        print('inside screwspeed_update')#z,screwspeed_epsilon,screwspeed_etha,screwspeed_fin
        if self.z==1:
            if self.screwspeed_epsilon>80:
                print("we should go to solidcontent_update now:",self.screwspeed_epsilon)
                self.solidcontent_update()
            else:
                
                self.screwspeed=self.screwspeed_fin+self.screwspeed_epsilon
                print("final1 self.screwspeed to enter solidcontent_update:",self.screwspeed)
                print(self.z)
                self.screwspeed_epsilon=self.screwspeed_epsilon+20
                self.z=2
            
        else:
            if self.screwspeed_etha>80:
                print("we should go to solidcontent_update now:",self.screwspeed_etha)
                self.solidcontent_update()
            else:
                
                self.screwspeed=self.screwspeed_fin-self.screwspeed_etha
                print("final self.screwspeed to enter solidcontent_update:",self.screwspeed)
                self.screwspeed_etha=self.screwspeed_etha+20
                self.z=1
           
            
    def solidcontent_update(self):#x,solidcontent_epsilon,solidcontent_etha,solidcontent_fin
        print('inside solidcontent_update')
        print("screwspeed_fin+",self.screwspeed_fin)
        if self.x==1:
            if self.solidcontent_epsilon>5:
                print("Divergence occured!")
                exit()
            self.solidcontent=self.solidcontent_fin+self.solidcontent_epsilon
            self.solidcontent_epsilon=self.solidcontent_epsilon+1 
            self.screwspeed_epsilon=20
            self.screwspeed_etha=20
            self.x=2
            self.z=1
            self.screwspeed_update()           
        else:
            if self.solidcontent_etha>5:
                print("Divergence occured!")
                exit()
                
            self.solidcontentt=self.solidcontent_fin-self.solidcontent_etha
            self.solidcontent_etha=self.solidcontent_etha+1
            self.screwspeed_epsilon=20
            self.screwspeed_etha=20
            self.x=1
            self.screwspeed_update()
            
            
#neuroController function gets massflow_ref, Energy_ref, Torque_ref, Energy_pred, Torque_pred
#and returns[self.Delta_screwspeed_pred,self.Delta_solidcontroler_pred]          
    def neuroController(self):                 
        print('inside neurocontroller')
        # prepare inputs 
        input_NC=self.scaler_nc_input.transform(np.array([[self.massflow_ref, self.Energy_ref, self.Torque_ref,self.Energy_pred,self.Torque_pred]], dtype=object))
        #predict Delta_screwspeed_pred
        
        Delta_screwspeed_pred=pd.DataFrame(self.model_Dn_Neurocontroller.predict(input_NC))
        #predict cm_Neurocontroller
        
        Delta_solidcontent_pred = pd.DataFrame(self.model_cm_Neurocontroller.predict(input_NC))
    
        self.Delta_screwspeed_pred=self.model_Dn_Neurocontroller.predict(input_NC)
        self.Delta_solidcontent_pred=self.model_cm_Neurocontroller.predict(input_NC)
        print(self.Delta_screwspeed_pred,self.Delta_solidcontent_pred)        

#updat function get the Delta_screwspeed_pred,Delta_solidcontent_pred
#and add them to the previous screwspeed and solidcontroler so it 
#returnes new screwspeed, solidcontroler values
    def update(self):
        print('inside update')
        self.screwspeed=self.screwspeed+self.Delta_screwspeed_pred
        self.solidcontent=self.solidcontent+self.Delta_solidcontent_pred
        print(self.screwspeed, self.solidcontent)
        
    def start_process(self):
        self.plot_array=np.array([[]])
        while True:
            j=1
            
            self.Extruder()
            if self.coarse_criterion():
                while True:
                    if j >15:
                        break
                
                    if self.fine_criterion():
                        break
                    else:
                        self.screwspeed_update()
                        print(j)
                        j = j+1
                        self.Extruder()
                print('everythinh is in range and final screwspeed and final solidcontent are {} and {} respectively'.format(screwspeed,solidcontent))
                break
            else:
                self.neuroController()
                self.update()
#plt.scatter(self.plot_array[:,0],self.plot_array[:,1])     
            time.sleep(3)
            
            
            
        
        

        

    

In [31]:
screwspeed,solidcontent,massflow_ref,Energy_ref,Torque_ref= 400,0,2.28,6552,2.972

In [35]:
m = load("model_Energy.h5")

c:\Users\aminr\anaconda3\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator DummyRegressor from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\aminr\anaconda3\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\aminr\anaconda3\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GradientBoostingRegressor from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. U

In [36]:
m.predict([[2.28,400 , 0]])

array([2608.2716424])

In [30]:
scaler = joblib.load("scaler_ext.joblib")
m.predict(scaler.transform([[2.28,480 , 70]]))

c:\Users\aminr\anaconda3\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\aminr\anaconda3\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


array([6584.0495237])

In [32]:
Extruder_Controller=ExtruderController(screwspeed,solidcontent,massflow_ref,Energy_ref,Torque_ref)

c:\Users\aminr\anaconda3\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator DummyRegressor from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\aminr\anaconda3\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\aminr\anaconda3\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GradientBoostingRegressor from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. U

In [34]:
Extruder_Controller.start_process()

inside Extruder
screwspeed:480, solidcontent:1
[6584.0495237] [2.92817026]
inside coarse criterion
inside fine criterion
inside screwspeed_update
final self.screwspeed to enter solidcontent_update: 320
1
inside Extruder
screwspeed:320, solidcontent:1
[6584.0495237] [2.92817026]
inside fine criterion
inside screwspeed_update
we should go to solidcontent_update now: 100
inside solidcontent_update
screwspeed_fin+ 400
inside screwspeed_update
final1 self.screwspeed to enter solidcontent_update: 420
1
2
inside Extruder
screwspeed:420, solidcontent:1
[6584.0495237] [2.92817026]
inside fine criterion
inside screwspeed_update
final self.screwspeed to enter solidcontent_update: 380
3
inside Extruder
screwspeed:380, solidcontent:1
[6584.0495237] [2.92817026]
inside fine criterion
inside screwspeed_update
final1 self.screwspeed to enter solidcontent_update: 440
1
4
inside Extruder
screwspeed:440, solidcontent:1
[6584.0495237] [2.92817026]
inside fine criterion
inside screwspeed_update
final self.

c:\Users\aminr\anaconda3\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\aminr\anaconda3\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\aminr\anaconda3\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\aminr\anaconda3\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\aminr\anaconda3\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\aminr\anaconda3\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature nam

In [11]:
massflow_ref=3
screwspeed=900
solidcontent=80
#prepare inputs 
input_Extruder=np.array([[(massflow_ref),(screwspeed),(solidcontent)]], dtype=object)

#predict Energy-pred
model_Energy=load('model_Energy.h5') 
Energy_pred = pd.DataFrame(model_Energy.predict(input_Extruder))
        
#predict Torque_pred
model_Torque=load('model_Torque.h5') 
Torque_pred = pd.DataFrame(model_Torque.predict(input_Extruder))

print(Energy_pred, Torque_pred)
#plot_array=np.array([[Nplant],[Wplant],[SME_pred[0]], [T_pred[0]]])
plot_array=np.array([])
plot_array= np.append(plot_array,[[screwspeed],[solidcontent],[Energy_pred], [Torque_pred]] )

plot_array

             0
0  5829.037787            0
0  10.585044


C:\Users\Mahtab\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:1859: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asanyarray(a).ravel(order=order)


array([900, 80,              0
                0  5829.037787,            0
                                0  10.585044], dtype=object)

In [12]:
Extruder_Controller.start_process()

inside Extruder
screwspeed:[4296.18001315], solidcontent:[68.54349632]
[5829.03778661] [10.58504427]
inside coarse criterion
inside neurocontroller
[36.07574086] [-0.01348615]
inside update
[4332.25575401] [68.53001018]
inside Extruder
screwspeed:[4332.25575401], solidcontent:[68.53001018]
[5829.03778661] [10.58504427]
inside coarse criterion
inside neurocontroller
[36.07574086] [-0.01348615]
inside update
[4368.33149487] [68.51652403]
inside Extruder
screwspeed:[4368.33149487], solidcontent:[68.51652403]
[5829.03778661] [10.58504427]
inside coarse criterion
inside neurocontroller
[36.07574086] [-0.01348615]
inside update
[4404.40723573] [68.50303789]
inside Extruder
screwspeed:[4404.40723573], solidcontent:[68.50303789]
[5829.03778661] [10.58504427]
inside coarse criterion
inside neurocontroller
[36.07574086] [-0.01348615]
inside update
[4440.4829766] [68.48955174]
inside Extruder
screwspeed:[4440.4829766], solidcontent:[68.48955174]
[5829.03778661] [10.58504427]
inside coarse criteri

KeyboardInterrupt: 

In [13]:
Energy= np.array(Extruder_Controller.Energy_list).ravel()
Torque= np.array(Extruder_Controller.Torque_list).ravel()
Screwspeed= np.array(Extruder_Controller.screwspeed_list).ravel()
Solidcontent= np.array(Extruder_Controller.solidcontent_list).ravel()
iteration=list(range(len(Energy)))

<ipython-input-13-f669828de54b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Screwspeed= np.array(Extruder_Controller.screwspeed_list).ravel()
<ipython-input-13-f669828de54b>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Solidcontent= np.array(Extruder_Controller.solidcontent_list).ravel()


In [14]:
Energy

array([5829.03778661, 5829.03778661, 5829.03778661, 5829.03778661,
       5829.03778661, 5829.03778661, 5829.03778661, 5829.03778661,
       5829.03778661, 5829.03778661, 5829.03778661, 5829.03778661,
       5829.03778661, 5829.03778661, 5829.03778661, 5829.03778661,
       5829.03778661, 5829.03778661, 5829.03778661, 5829.03778661,
       5829.03778661, 5829.03778661, 5829.03778661, 5829.03778661,
       5829.03778661, 5829.03778661, 5829.03778661, 5829.03778661,
       5829.03778661, 5829.03778661, 5829.03778661, 5829.03778661,
       5829.03778661, 5829.03778661, 5829.03778661, 5829.03778661,
       5829.03778661, 5829.03778661, 5829.03778661, 5829.03778661,
       5829.03778661, 5829.03778661, 5829.03778661, 5829.03778661,
       5829.03778661, 5829.03778661, 5829.03778661, 5829.03778661,
       5829.03778661, 5829.03778661, 5829.03778661, 5829.03778661,
       5829.03778661, 5829.03778661, 5829.03778661, 5829.03778661,
       5829.03778661, 5829.03778661, 5829.03778661, 5829.03778